In [39]:
import polars as pl
import numpy as np
import numpy.typing as npt
from sklearn.model_selection import train_test_split, cross_val_score

In [40]:
# constants

TEST_SIZE: np.float16 = 0.3
CROSS_VALIDATION_K: np.int8 = 5
RANDOM_STATE: np.int8 = 42

In [41]:
COLUMNS: list[str] = ["tl", "tm", "tr", "ml", "mm", "mr", "bl", "bm", "br", "class"]

# ttt is a acronym for tic tac toe
ttt_df: pl.DataFrame = pl.read_csv("../data/tic-tac-toe.data", has_header=False, new_columns=COLUMNS)
ttt_df.head()

tl,tm,tr,ml,mm,mr,bl,bm,br,class
str,str,str,str,str,str,str,str,str,str
"""x""","""x""","""x""","""x""","""o""","""o""","""x""","""o""","""o""","""positive"""
"""x""","""x""","""x""","""x""","""o""","""o""","""o""","""x""","""o""","""positive"""
"""x""","""x""","""x""","""x""","""o""","""o""","""o""","""o""","""x""","""positive"""
"""x""","""x""","""x""","""x""","""o""","""o""","""o""","""b""","""b""","""positive"""
"""x""","""x""","""x""","""x""","""o""","""o""","""b""","""o""","""b""","""positive"""


In [42]:
# splitting data

data: npt.NDArray[np.string_] = ttt_df.to_numpy()
ttt_X: npt.NDArray[np.string_] = data[:, 0:data.shape[1] - 1]
ttt_y: npt.NDArray[np.string_] = data[:, data.shape[1] - 1]
X_train, X_test, y_train, y_test = train_test_split(ttt_X, ttt_y, test_size=TEST_SIZE, random_state=RANDOM_STATE)